In [ ]:
%%capture
!pip install google-colab-selenium[undetected]

In [ ]:
from bs4 import BeautifulSoup
import google_colab_selenium as gs
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import time
import random
import requests
from tqdm import tqdm
import json
# import undetected_chromedriver.v2 as uc
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from google.colab import files


In [ ]:
# Do you need to use a proxy?
PROXY = False

In [ ]:
def validate_proxy(proxy):
    try:
        response = requests.get('http://www.google.com', proxies={'http': proxy, 'https': proxy}, timeout=5)
        return response.status_code == 200
    except:
        return False

In [ ]:
if PROXY:
  with open('proxies.txt', 'r') as file:
      proxies = [line.strip() for line in file.readlines()]

  num_proxies_needed = 1
  valid_proxies = []

  for proxy in tqdm(proxies):
    if len(valid_proxies) >= num_proxies_needed:
      break
    if validate_proxy(proxy):
      valid_proxies.append(proxy)
      print(f"\nValidated {len(valid_proxies)} / {num_proxies_needed} proxies")

  print(f"Validated {len(valid_proxies)} proxies")

In [ ]:
if PROXY:
  chrome_options = webdriver.ChromeOptions()
  proxy = random.choice(valid_proxies)
  chrome_options.add_argument('--disable-blink-features=AutomationControlled')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument('--headless')
  chrome_options.add_argument(f'--proxy-server=http://{proxy}')
  # return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
  driver = gs.UndetectedChrome(options=chrome_options)
else:
  driver = gs.UndetectedChrome()

<IPython.core.display.Javascript object>

In [ ]:
# list_url is the main page that includes music links
list_url = "https://www.ultimate-guitar.com/explore"
# page_url is the url (start) to other pages with music links. the end of the url will say, page=1, page=2, etc.
page_url = "https://www.ultimate-guitar.com/explore?page="

In [ ]:
# get the chords to ONE page / song
def get_chords(html):
  soup = BeautifulSoup(html, 'html.parser')
  spans = soup.findAll("span", class_= "fciXY _Oy28")
  chords = []
  for span in spans:
    if span.get('data-name') is not None:
      chords.append(span.get('data-name'))
  return chords

In [ ]:
def random_delay(min_seconds=1, max_seconds=7):
    time.sleep(random.uniform(min_seconds, max_seconds))

def simulate_mouse_movements(driver):
    action = ActionChains(driver)
    element = driver.find_element(By.TAG_NAME, 'body')
    action.move_to_element_with_offset(element, random.randint(0, 100), random.randint(0, 100))
    action.perform()
    random_delay(0.5, 2)

def simulate_scrolling(driver):
    body = driver.find_element(By.TAG_NAME, 'body')
    for _ in range(random.randint(1, 3)):
        body.send_keys(Keys.PAGE_DOWN)
        random_delay(1, 3)

def simulate_typing(driver, element, text):
    for char in text:
        element.send_keys(char)
        random_delay(0.1, 0.3)

In [ ]:
# url = 'https://tabs.ultimate-guitar.com/tab/frank-sinatra/somethin-stupid-chords-831238'
# try:
#   driver.get(url)
#   try:
#     WebDriverWait(driver, 10).until(lambda driver: "Just a moment..." not in driver.title)
#   except TimeoutException:
#     print(f"Pages stopped loading") # this should notify you how many were scraped before we got blocked.
# except Exception as e:
#   print("Driver could not connect to page")
#   print(e)
# html = driver.execute_script("return document.body.innerHTML;")
# soup = BeautifulSoup(html, 'html.parser')
# try:
#   search_box = driver.find_element(By.CSS_SELECTOR, '.ABzEe.YV1G9 input[type="text"]')
#   # search_box.send_keys("Taylor Swift")
#   simulate_typing(driver, search_box, "Taylor Swift")
#   search_box.send_keys(Keys.RETURN)
# except Exception as e:
#   print("error with search box")
#   print(e)
# print(get_chords(html))

In [ ]:
# we are either adding to tab_urls or using a list of tab_urls that already exist. these urls are links to lots of songs
tab_urls = []
try:
  with open('tab_urls.json', 'r') as f:
    tab_urls_string = f.read()
  tab_urls = json.loads(tab_urls_string)
  print(f"there are {len(tab_urls)} tab urls:")
  print(tab_urls) # delete this line once the readlines() code is verified
except FileNotFoundError:
  # if the tab_urls.json is not found, we will scrape the tab urls from scratch.
  print("rewriting tab_urls.json")
  for i in range(10,20):
    try:
      driver.get(page_url + str(i))
    except Exception as e:
      print("Driver could not connect to page")
      print(e)
      break
    html_source_code = driver.execute_script("return document.body.innerHTML;")
    soup = BeautifulSoup(html_source_code, 'html.parser')
    links = soup.findAll("a")
    for link in links:
      link_text = link.get("href")
      if "/tab/" in link_text:
        tab_urls.append(link_text)
  with open('tab_urls.json', 'w') as f:
    json.dump(tab_urls, f)

rewriting tab_urls.json


In [ ]:
tab_urls

['https://tabs.ultimate-guitar.com/tab/joni-mitchell/both-sides-now-chords-1140752',
 'https://tabs.ultimate-guitar.com/tab/zach-top/i-never-lie-chords-5212647',
 'https://tabs.ultimate-guitar.com/tab/misc-cartoons/spongebob-squarepants-ending-theme-tabs-116579',
 'https://tabs.ultimate-guitar.com/tab/john-mayer/slow-dancing-in-a-burning-room-chords-706621',
 'https://tabs.ultimate-guitar.com/tab/james-blunt/youre-beautiful-chords-179027',
 'https://tabs.ultimate-guitar.com/tab/lynyrd-skynyrd/free-bird-chords-1045075',
 'https://tabs.ultimate-guitar.com/tab/shawn-mendes/stitches-chords-1721601',
 'https://tabs.ultimate-guitar.com/tab/seafret/atlantis-chords-1733648',
 'https://tabs.ultimate-guitar.com/tab/tears-for-fears/everybody-wants-to-rule-the-world-chords-1503308',
 'https://tabs.ultimate-guitar.com/tab/david-kushner/daylight-chords-4695719',
 'https://tabs.ultimate-guitar.com/tab/lukas-graham/7-years-chords-1752864',
 'https://tabs.ultimate-guitar.com/tab/acdc/highway-to-hell-ch

In [ ]:
try:
  with open('song_chords.json', 'r') as f:
    existing_songs = json.load(f) # parse what we already have and read it into a list. This will allow us to skip already scraped ones.
    print(f"there are {len(existing_songs)} songs found in file songs_chords.json")
  id_existing_songs = [song['id'] for song in existing_songs]
except FileNotFoundError:
  print("no file named song_chords.json, no chords found")
  existing_songs1 = []
  id_existing_songs = []


there are 1127 songs found in file songs_chords.json


In [ ]:
i = 1

for tab_url in tqdm(tab_urls):
  if tab_url.split("/")[-1] in id_existing_songs:
    print(f"song already exists: skipping {tab_url}")
    continue
  try:
    driver.get(tab_url)

    # page loading
    try:
      WebDriverWait(driver, 10).until(lambda driver: "Just a moment..." not in driver.title)
    except TimeoutException:
      print(f"Pages stopped loading at {i}") # this should notify you how many were scraped before we got blocked.
      break

    html = driver.execute_script("return document.body.innerHTML;")

    # simulate human behavior
    # if i % 5 == 0:
    #   try:
    #       # Wait until the element is present
    #       search_box = WebDriverWait(driver, 10).until(
    #           EC.presence_of_element_located((By.CSS_SELECTOR, '.ABzEe.YV1G9 input[type="text"]'))
    #       )

    #       # Simulate typing
    #       simulate_typing(driver, search_box, 'Taylor Swift')
    #       search_box.send_keys(Keys.RETURN)

    #   except Exception as e:
    #     print("error with search box")
    #     print(f"Error: {e}")

    # random_delay(2, 4)
    # simulate_mouse_movements(driver)
    # random_delay(2, 7)
    # simulate_scrolling(driver)

    # extract song information
    song = {
        "song_number": i + len(existing_songs),
        "id": tab_url.split("/")[-1],
        "name": driver.title,
        "chords": get_chords(html)
    }

    if len(song['chords']) > 300:
      print("Album chords found- skipping")
      continue
    if len(song['chords']) == 0:
      print("No chords found- skipping")
      continue

    # store song in array
    existing_songs.append(song)
    i += 1

  # driver error
  except Exception as e:
    print(e)

In [ ]:
with open('song_chords.json', 'w') as file:
  json.dump(existing_songs, file)

In [ ]:
print(f"There are now {len(existing_songs)} songs in the list. Example of last item: {existing_songs[-2]}")

There are now 1180 songs in the list. Example of last item: {'song_number': 1230, 'id': 'gravity-chords-641307', 'name': 'GRAVITY CHORDS by John Mayer @ Ultimate-Guitar.Com', 'chords': ['G', 'C', 'G', 'C', 'G', 'C', 'G', 'C']}


In [ ]:
files.download('song_chords.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>